```
% dmcopy src.arf "arf.csv[opt kernel=text,sep=',']"
% head -5 arf.csv 
#TEXT/SIMPLE
#,ENERG_LO,ENERG_HI,SPECRESP
0.3000000, 0.3100000, 4.874343
0.3100000, 0.3200000, 14.82926
0.3200000, 0.3300000, 21.30229
```

So manually edit to remove the first line and fix up the column names:

```
% head -5 arf.csv 
ENERG_LO,ENERG_HI,SPECRESP
0.3000000, 0.3100000, 4.874343
0.3100000, 0.3200000, 14.82926
0.3200000, 0.3300000, 21.30229
0.3300000, 0.3400000, 28.51495
```

Will the spaces cause a problem?

Also

```
% git clone https://github.com/acowley/Frames.git
% cabal sandbox add-source Frames
% cabal install Frames foldl lens-family --dry-run
% cabal install Frames foldl lens-family
```

In [1]:
-- from http://acowley.github.io/Frames/

{-# LANGUAGE ConstraintKinds, DataKinds, FlexibleContexts, GADTs,
             OverloadedStrings, PatternSynonyms, QuasiQuotes,
             ScopedTypeVariables, TemplateHaskell, TypeOperators,
             ViewPatterns #-}

:set -XConstraintKinds
:set -XDataKinds
:set -XFlexibleContexts
:set -XGADTs
:set -XOverloadedStrings
:set -XPatternSynonyms
:set -XQuasiQuotes
:set -XScopedTypeVariables
:set -XTemplateHaskell
:set -XTypeOperators
:set -XViewPatterns

Apparently the `foldl` package is quite important to the processing...

In [45]:
{-
import Control.Applicative
-- import qualified Control.Foldl as L  -- for some reason this won't load, so comment out for now
import qualified Data.Foldable as F
import Data.Proxy (Proxy(..))
-- import Lens.Family -- as does this (presumably these are packages external to frames requirements)
import Frames
import Frames.CSV (readTableOpt, rowGen, RowGen(..))
import Pipes hiding (Proxy)
import qualified Pipes.Prelude as P
-}

import qualified Control.Foldl as L
import qualified Data.Foldable as F
import qualified Pipes.Prelude as P

import Control.Applicative

import Lens.Family

import Frames
import Frames.CSV (readTableOpt)

import Pipes hiding (Proxy)

In [3]:
tableTypes "ARF" "../data/arf.csv"

In [4]:
:opt no-pager

In [5]:
:info ARF

type ARF = Record '["ENERG_LO" :-> Double, "ENERG_HI" :-> Double, "SPECRESP" :-> Double] -- Defined at :1:1

Cool. This looks promising. As well as impenetrable.

The de-capitalisation is apparently something that's going to be common in this
notebook!

In [6]:
:t aRFParser

aRFParser :: ParserOptions

In [7]:
arfStream = readTableOpt aRFParser "../data/arf.csv"

In [8]:
:type arfStream

arfStream :: forall (m :: * -> *) (rs :: [*]). (ReadRec rs, MonadIO m) => Producer (Record rs) m ()

If you say so... Let's convert it into an in-memory type, as it's a small dataset.

In [9]:
-- signature needed for the type inference
arf <- inCoreAoS arfStream :: IO (Frame ARF)

In [10]:
:type arf

arf :: Frame ARF

In [15]:
:type view

view :: forall b a a' b'. FoldLike b a a' b b' -> a -> b

In [17]:
:type view sPECRESP

view sPECRESP :: forall (rs0 :: [*]). RElem SPECRESP rs0 (RIndex SPECRESP rs0) => Record rs0 -> Double

In [18]:
:type view sPECRESP <$> arf

view sPECRESP <$> arf :: Frame Double

In [19]:
view sPECRESP <$> arf

No instance for (Show (Frame Double)) arising from a use of ‘print’
In a stmt of an interactive GHCi command: print it

In [24]:
:type L.fold
:type L.fold L.minimum
:type L.fold L.minimum (view sPECRESP <$> arf)

L.fold :: forall a b (f :: * -> *). Foldable f => Fold a b -> f a -> b

L.fold L.minimum :: forall a (f :: * -> *). (Foldable f, Ord a) => f a -> Maybe a

L.fold L.minimum (view sPECRESP <$> arf) :: Maybe Double

In [25]:
L.fold L.minimum (view sPECRESP <$> arf)
L.fold L.maximum (view sPECRESP <$> arf)

Just 0.5657126

Just 672.1996

In [26]:
-- from the tutorial
minMax :: Ord a => L.Fold a (Maybe a, Maybe a)
minMax = (,) <$> L.minimum <*> L.maximum

In [27]:
L.fold minMax (view sPECRESP <$> arf)

(Just 0.5657126,Just 672.1996)

In [34]:
:type L.pretraverse sPECRESP

L.pretraverse sPECRESP :: forall r (rs0 :: [*]). RElem SPECRESP rs0 (RIndex SPECRESP rs0) => Fold Double r -> Fold (Record rs0) r

In [36]:
:type L.pretraverse sPECRESP minMax 

L.pretraverse sPECRESP minMax :: forall (rs0 :: [*]). RElem SPECRESP rs0 (RIndex SPECRESP rs0) => Fold (Record rs0) (Maybe Double, Maybe Double)

In [39]:
L.fold (L.pretraverse sPECRESP minMax) arf

(Just 0.5657126,Just 672.1996)

In [42]:
mapM_ print (take 3 (F.toList arf))

{ENERG_LO :-> 0.3, ENERG_HI :-> 0.31, SPECRESP :-> 4.874343}
{ENERG_LO :-> 0.31, ENERG_HI :-> 0.32, SPECRESP :-> 14.82926}
{ENERG_LO :-> 0.32, ENERG_HI :-> 0.33, SPECRESP :-> 21.30229}

In [44]:
nrows = frameLength arf
mapM_ (print . frameRow arf) [nrows - 3 .. nrows - 1]

{ENERG_LO :-> 10.97, ENERG_HI :-> 10.98, SPECRESP :-> 0.6049552}
{ENERG_LO :-> 10.98, ENERG_HI :-> 10.99, SPECRESP :-> 0.5852773}
{ENERG_LO :-> 10.99, ENERG_HI :-> 11.0, SPECRESP :-> 0.5657126}

In [46]:
-- can I select rows where SPECRESP > 600, say? Pipes to the rescue,
-- but does this mean losing the in-memory representation?

pipePreview arfStream 6 (P.filter ((> 600) . view sPECRESP))

Could not deduce (Data.Vinyl.TypeLevel.RecAll Data.Vinyl.Functor.Identity rs0 Show) arising from a use of ‘pipePreview’
from the context (MonadIO m) bound by the inferred type of it :: MonadIO m => m () at <interactive>:1:1-60
In the expression: pipePreview arfStream 6 (P.filter ((> 600) . view sPECRESP))
In an equation for ‘it’: it = pipePreview arfStream 6 (P.filter ((> 600) . view sPECRESP))

Could not deduce (Frames.CSV.ReadRec rs0) arising from a use of ‘arfStream’
from the context (MonadIO m) bound by the inferred type of it :: MonadIO m => m () at <interactive>:1:1-60
The type variable ‘rs0’ is ambiguous
Note: there are several potential instances:
  instance (Data.Readable.Readable t, Frames.CSV.ReadRec ts) => Frames.CSV.ReadRec ((s :-> t) : ts) -- Defined in ‘Frames.CSV’
  instance Frames.CSV.ReadRec '[] -- Defined in ‘Frames.CSV’
In the first argument of ‘pipePreview’, namely ‘arfStream’
In the expression: pipePreview arfStream 6 (P.filter ((> 600) . view sPECRESP))
In an equation for ‘it’: it = pipePreview arfStream 6 (P.filter ((> 600) . view sPECRESP))

Could not deduce (RElem SPECRESP rs0 (Data.Vinyl.TypeLevel.RIndex SPECRESP rs0)) arising from a use of ‘sPECRESP’
from the context (MonadIO m) bound by the inferred type of it :: MonadIO m => m () at <interactive>:1:1-60
The type variable ‘rs0’ is ambiguous
Note: there are several potential instances:
  instance (Data.Vinyl.TypeLevel.RIndex r (s : rs) ~ 'Data.Vinyl.TypeLevel.S i, RElem r rs i) => RElem r (s : rs) ('Data.Vinyl.TypeLevel.S i) -- Defined in ‘Data.Vinyl.Lens’
  instance RElem r (r : rs) 'Data.Vinyl.TypeLevel.Z -- Defined in ‘Data.Vinyl.Lens’
In the first argument of ‘view’, namely ‘sPECRESP’
In the second argument of ‘(.)’, namely ‘view sPECRESP’
In the first argument of ‘P.filter’, namely ‘((> 600) . view sPECRESP)’

In [49]:
areaRows :: (SPECRESP ∈ rs, Monad m) => Pipe (Record rs) (Record rs) m r
areaRows = P.filter ((> 600) . view sPECRESP)

In [51]:
-- AHA: it's probably because I did not type arfStream enough earlier...
pipePreview (arfStream :: Producer ARF IO ()) 6 areaRows

{ENERG_LO :-> 1.18, ENERG_HI :-> 1.19, SPECRESP :-> 601.2171}
{ENERG_LO :-> 1.19, ENERG_HI :-> 1.2, SPECRESP :-> 604.2618}
{ENERG_LO :-> 1.2, ENERG_HI :-> 1.21, SPECRESP :-> 607.2027}
{ENERG_LO :-> 1.21, ENERG_HI :-> 1.22, SPECRESP :-> 610.0424}
{ENERG_LO :-> 1.22, ENERG_HI :-> 1.23, SPECRESP :-> 612.9407}
{ENERG_LO :-> 1.23, ENERG_HI :-> 1.24, SPECRESP :-> 615.865}

In [52]:
pipePreview (arfStream :: Producer ARF IO ()) 6 (P.filter ((> 600) . view sPECRESP))

{ENERG_LO :-> 1.18, ENERG_HI :-> 1.19, SPECRESP :-> 601.2171}
{ENERG_LO :-> 1.19, ENERG_HI :-> 1.2, SPECRESP :-> 604.2618}
{ENERG_LO :-> 1.2, ENERG_HI :-> 1.21, SPECRESP :-> 607.2027}
{ENERG_LO :-> 1.21, ENERG_HI :-> 1.22, SPECRESP :-> 610.0424}
{ENERG_LO :-> 1.22, ENERG_HI :-> 1.23, SPECRESP :-> 612.9407}
{ENERG_LO :-> 1.23, ENERG_HI :-> 1.24, SPECRESP :-> 615.865}

In [53]:
-- would be nice to have a HTML representation of the table.
arf

No instance for (Show (Frame ARF)) arising from a use of ‘print’
In a stmt of an interactive GHCi command: print it

In [55]:
-- and how to extract the data to plot it? Frames has plot examples